In [1]:
import cv2
import pyaudio
import numpy as np
import platform
import os
import sys

def obtener_info_sistema():
    """Obtiene información básica del sistema para contexto"""
    info = {
        "sistema_operativo": platform.system(),
        "version": platform.version(),
        "arquitectura": platform.architecture(),
        "python_version": sys.version,
        "opencv_version": cv2.__version__,
        "pyaudio_version": pyaudio.__version__
    }
    
    print("\n===== INFORMACIÓN DEL SISTEMA =====")
    for clave, valor in info.items():
        print(f"{clave.replace('_', ' ').title()}: {valor}")
    
    return info

def obtener_info_microfonos():
    """Obtiene información detallada de todos los micrófonos disponibles"""
    print("\n===== INFORMACIÓN DE MICRÓFONOS =====")
    
    try:
        p = pyaudio.PyAudio()
        info_host = p.get_host_api_info_by_index(0)
        num_dispositivos = info_host.get('deviceCount')
        
        print(f"Número total de dispositivos de audio: {num_dispositivos}")
        print(f"API de audio predeterminada: {info_host.get('name')}")
        
        # Obtener información sobre los hosts de audio
        num_apis = p.get_host_api_count()
        print(f"\nAPIs de audio disponibles: {num_apis}")
        for i in range(num_apis):
            api_info = p.get_host_api_info_by_index(i)
            print(f"  API {i}: {api_info.get('name')}")
        
        # Listar todos los dispositivos de entrada (micrófonos)
        print("\nMICRÓFONOS DETECTADOS:")
        
        microfonos = []
        for i in range(num_dispositivos):
            try:
                info_dispositivo = p.get_device_info_by_index(i)
                
                # Sólo mostrar dispositivos de entrada
                if info_dispositivo.get('maxInputChannels') > 0:
                    # Recopilar información detallada
                    mic_info = {
                        'id': i,
                        'nombre': info_dispositivo.get('name'),
                        'api_host': info_dispositivo.get('hostApi'),
                        'api_nombre': p.get_host_api_info_by_index(info_dispositivo.get('hostApi')).get('name'),
                        'canales_entrada': info_dispositivo.get('maxInputChannels'),
                        'canales_salida': info_dispositivo.get('maxOutputChannels'),
                        'tasa_muestreo_defecto': info_dispositivo.get('defaultSampleRate'),
                        'latencia_entrada_baja': info_dispositivo.get('defaultLowInputLatency'),
                        'latencia_entrada_alta': info_dispositivo.get('defaultHighInputLatency'),
                        'es_predeterminado': info_host.get('defaultInputDevice') == i
                    }
                    
                    microfonos.append(mic_info)
                    
                    # Mostrar información en consola
                    print(f"\n-- Micrófono {i}: {mic_info['nombre']} --")
                    print(f"  API: {mic_info['api_nombre']}")
                    print(f"  Canales de entrada: {mic_info['canales_entrada']}")
                    print(f"  Tasa de muestreo: {mic_info['tasa_muestreo_defecto']} Hz")
                    if mic_info['es_predeterminado']:
                        print("  ⭐ DISPOSITIVO DE ENTRADA PREDETERMINADO")
                    
                    # Verificar si parece ser un micrófono USB
                    es_usb = "usb" in mic_info['nombre'].lower() or "camera" in mic_info['nombre'].lower()
                    if es_usb:
                        print("  📷 Posible micrófono de cámara USB")
            
            except Exception as e:
                print(f"Error al acceder al dispositivo {i}: {e}")
        
        p.terminate()
        return microfonos
    
    except Exception as e:
        print(f"Error al enumerar micrófonos: {e}")
        return []

def obtener_info_camaras():
    """Obtiene información detallada de todas las cámaras disponibles"""
    print("\n===== INFORMACIÓN DE CÁMARAS =====")
    
    # Variables para determinar el número máximo de cámaras a probar
    max_camaras = 10  # Ajusta este valor según sea necesario
    camaras = []
    
    print(f"Buscando cámaras (probando índices del 0 al {max_camaras-1})...")
    
    for i in range(max_camaras):
        try:
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                # Obtener propiedades de la cámara
                ancho = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                alto = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                fps = cap.get(cv2.CAP_PROP_FPS)
                codec = int(cap.get(cv2.CAP_PROP_FOURCC))
                
                # Formatear el codec para hacerlo legible
                codec_str = "".join([chr((codec >> 8 * i) & 0xFF) for i in range(4)])
                
                # Verificar si se pueden leer frames
                ret, frame = cap.read()
                puede_leer_frames = ret
                
                # Otras propiedades interesantes
                brillo = cap.get(cv2.CAP_PROP_BRIGHTNESS)
                contraste = cap.get(cv2.CAP_PROP_CONTRAST)
                saturacion = cap.get(cv2.CAP_PROP_SATURATION)
                
                # Guardar información
                camara_info = {
                    'id': i,
                    'resolucion': f"{ancho}x{alto}",
                    'ancho': ancho,
                    'alto': alto,
                    'fps': fps,
                    'codec': codec_str,
                    'puede_leer_frames': puede_leer_frames,
                    'brillo': brillo,
                    'contraste': contraste,
                    'saturacion': saturacion
                }
                
                camaras.append(camara_info)
                
                # Mostrar información en consola
                print(f"\n-- Cámara {i} --")
                print(f"  Resolución: {ancho}x{alto}")
                print(f"  FPS: {fps}")
                print(f"  Formato: {codec_str}")
                
                # Verificar si la cámara funciona correctamente
                if puede_leer_frames:
                    print("  ✅ La cámara puede capturar frames")
                else:
                    print("  ❌ La cámara no puede capturar frames correctamente")
                
                # Mostrar dimensiones del frame (si está disponible)
                if puede_leer_frames and frame is not None:
                    print(f"  Dimensiones del frame capturado: {frame.shape}")
                
                # Intentar detectar si es una cámara USB
                # Nota: esto es una heurística y puede variar según el sistema
                es_posible_usb = (ancho <= 1280 and alto <= 720 and i > 0)
                if es_posible_usb:
                    print("  📷 Posible cámara USB externa")
                elif i == 0:
                    print("  💻 Posible cámara integrada")
                
                # Liberar la cámara
                cap.release()
            else:
                print(f"  Cámara {i}: No disponible o no se puede abrir")
        
        except Exception as e:
            print(f"  Error al acceder a la cámara {i}: {e}")
    
    print(f"\nTotal de cámaras detectadas: {len(camaras)}")
    return camaras

def correlacionar_camaras_microfonos(camaras, microfonos):
    """Intenta encontrar qué micrófonos corresponden a qué cámaras"""
    print("\n===== CORRELACIÓN ENTRE CÁMARAS Y MICRÓFONOS =====")
    
    pares = []
    
    # Buscar patrones en nombres de micrófonos que puedan indicar relación con cámaras
    for mic in microfonos:
        nombre_mic = mic['nombre'].lower()
        
        # Verificar si el micrófono parece pertenecer a una cámara
        es_mic_camara = any(palabra in nombre_mic for palabra in ['camera', 'cam', 'webcam', 'video', 'usb'])
        
        if es_mic_camara:
            # Intentar encontrar un número en el nombre que pueda corresponder a un ID
            posible_id_camara = None
            
            # Buscar patrones como "Camera 2" o "USB 3"
            for i in range(10):  # Números del 0 al 9
                if str(i) in nombre_mic:
                    posible_id_camara = i
                    break
            
            # Si no encontramos un número específico, usar heurísticas
            if posible_id_camara is None:
                # Para micrófonos de cámara, a menudo el ID será cercano
                posible_id_camara = mic['id'] - 1 if mic['id'] > 0 else 0
            
            # Verificar si existe una cámara con ese ID
            camara_correspondiente = next((cam for cam in camaras if cam['id'] == posible_id_camara), None)
            
            if camara_correspondiente:
                pares.append({
                    'microfono': mic,
                    'camara': camara_correspondiente,
                    'confianza': 'alta' if str(posible_id_camara) in nombre_mic else 'media'
                })
                print(f"\n✓ Posible emparejamiento:")
                print(f"  Micrófono: ID {mic['id']} - {mic['nombre']}")
                print(f"  Cámara: ID {camara_correspondiente['id']} - {camara_correspondiente['resolucion']}")
                print(f"  Nivel de confianza: {pares[-1]['confianza']}")
    
    # Si no encontramos pares por nombre, usar heurística de índices cercanos
    if not pares:
        print("\nNo se encontraron coincidencias claras por nombre.")
        print("Intentando emparejar por proximidad de índices...")
        
        for mic in microfonos:
            # Buscar cámaras con IDs cercanos
            for cam in camaras:
                # Heurística: Si el ID de cámara está a ±1 del ID del micrófono
                if abs(mic['id'] - cam['id']) <= 1:
                    pares.append({
                        'microfono': mic,
                        'camara': cam,
                        'confianza': 'baja'
                    })
                    print(f"\n✓ Posible emparejamiento por proximidad de índices:")
                    print(f"  Micrófono: ID {mic['id']} - {mic['nombre']}")
                    print(f"  Cámara: ID {cam['id']} - {cam['resolucion']}")
                    print(f"  Nivel de confianza: baja")
    
    if not pares:
        print("\nNo se pudieron determinar emparejamientos entre cámaras y micrófonos.")
    
    return pares

def mostrar_resumen_dispositivos(camaras, microfonos, pares):
    """Muestra un resumen de todos los dispositivos y posibles pares"""
    print("\n===== RESUMEN DE DISPOSITIVOS =====")
    
    # Resumen de cámaras
    print(f"\n🎥 CÁMARAS DETECTADAS: {len(camaras)}")
    for i, cam in enumerate(camaras):
        print(f"  {i+1}. Cámara {cam['id']}: {cam['resolucion']}, {cam['fps']} FPS")
    
    # Resumen de micrófonos
    print(f"\n🎤 MICRÓFONOS DETECTADOS: {len(microfonos)}")
    for i, mic in enumerate(microfonos):
        predeterminado = " (PREDETERMINADO)" if mic.get('es_predeterminado') else ""
        print(f"  {i+1}. Micrófono {mic['id']}: {mic['nombre']}{predeterminado}")
    
    # Resumen de pares
    print(f"\n🔗 POSIBLES PARES CÁMARA-MICRÓFONO: {len(pares)}")
    for i, par in enumerate(pares):
        cam = par['camara']
        mic = par['microfono']
        confianza = par['confianza']
        
        # Mostrar el nivel de confianza con emojis
        emoji_confianza = "✅" if confianza == 'alta' else "🟡" if confianza == 'media' else "⚠️"
        
        print(f"  {i+1}. {emoji_confianza} Cámara {cam['id']} + Micrófono {mic['id']} ({mic['nombre']})")
        print(f"     Confianza: {confianza.upper()}")

def main():
    """Función principal para ejecutar todo el análisis"""
    print("=" * 60)
    print("ANÁLISIS COMPLETO DE CÁMARAS Y MICRÓFONOS")
    print("=" * 60)
    
    # Obtener información del sistema
    info_sistema = obtener_info_sistema()
    
    # Obtener información de micrófonos
    microfonos = obtener_info_microfonos()
    
    # Obtener información de cámaras
    camaras = obtener_info_camaras()
    
    # Correlacionar cámaras y micrófonos
    pares = correlacionar_camaras_microfonos(camaras, microfonos)
    
    # Mostrar resumen
    mostrar_resumen_dispositivos(camaras, microfonos, pares)
    
    print("\n" + "=" * 60)
    print("ANÁLISIS COMPLETADO")
    print("=" * 60)
    
    return {
        'sistema': info_sistema,
        'microfonos': microfonos,
        'camaras': camaras,
        'pares': pares
    }

if __name__ == "__main__":
    resultado = main()

ANÁLISIS COMPLETO DE CÁMARAS Y MICRÓFONOS

===== INFORMACIÓN DEL SISTEMA =====
Sistema Operativo: Windows
Version: 10.0.22631
Arquitectura: ('64bit', 'WindowsPE')
Python Version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Opencv Version: 4.11.0
Pyaudio Version: 0.2.14

===== INFORMACIÓN DE MICRÓFONOS =====
Número total de dispositivos de audio: 7
API de audio predeterminada: MME

APIs de audio disponibles: 4
  API 0: MME
  API 1: Windows DirectSound
  API 2: Windows WASAPI
  API 3: Windows WDM-KS

MICRÓFONOS DETECTADOS:

-- Micrófono 0: Asignador de sonido Microsoft - Input --
  API: MME
  Canales de entrada: 2
  Tasa de muestreo: 44100.0 Hz

-- Micrófono 1: MicrÃ³fono (3- USB Camera) --
  API: MME
  Canales de entrada: 1
  Tasa de muestreo: 44100.0 Hz
  ⭐ DISPOSITIVO DE ENTRADA PREDETERMINADO
  📷 Posible micrófono de cámara USB

-- Micrófono 2: MicrÃ³fono (2- USB Camera) --
  API: MME
  Canales de entrada: 1
  Tasa de muestreo: 44100.0 Hz
  📷 Posi